In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("open_api_key")

## 청킹전략

### 부모-자식 분할

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [TextLoader(r"C:\Users\kimji\Desktop\ProgramFile\Study\How_to_invest_money.txt", encoding="utf-8-sig")]
docs = []
for loader in loaders:
    docs.extend(loader.load())

parent_splitter  = RecursiveCharacterTextSplitter(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings(api_key=api_key)
)

store = InMemoryStore()

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter = parent_splitter,
)


retriever.add_documents(docs)

print(f"부모 문서 수 : {len(list(store.yield_keys()))}")




부모 문서 수 : 219


In [7]:
query = "what are the tpyes of investments?"

retrieved_docs = retriever.get_relevant_documents(query)
print(f"부모 문서 : {retrieved_docs[0].page_content}")

C:\Users\kimji\AppData\Local\Temp\ipykernel_37428\3443923552.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


부모 문서 : There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.

Keeping these five general factors in mind, the present chapter will
discuss real-estate mortgages as a form of investment, both as adapted
to the requirements of private funds and of a business surplus.


In [8]:
sub_docs = vectorstore.similarity_search(query)
print(f"자식 문서 : {sub_docs[0].page_content}")

자식 문서 : forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.


## 질의 변형

### 다중 질의 생성